# Programming Assignment 4

Celio Bueri, Christoph Stelz

Exercise question

In the second example, the data are not linearly separable.

In [24]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])


## SVC Implementation

In [25]:
class SVCImpl(LinearClassifier):
    def __init__(self, n_iter=20, lr=0.01, regularizer=0.0, print_epoch_stats=True):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lr = lr
        self.regularizer = regularizer
        self.print_epoch_stats = print_epoch_stats

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # training algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                if y * score < 1:
                    self.w = (1 - self.regularizer * self.lr) * self.w + self.lr * y * x
                else:
                    self.w = (1 - self.regularizer * self.lr) * self.w
            if self.print_epoch_stats:
                print(f'epoch : {i}, train_loss : {self.score(X, Y)}')

    def score(self, X, Y):
        loss = 0.0
        min_weight = np.min(self.w.dot(self.w))
        for x, y in zip(X, self.encode_outputs(Y)):
            # loss for svc
            loss += max(0.0, 1.0 - y * x.dot(self.w))
        return self.regularizer / 2 * min_weight + loss / X.shape[0]

## Logistic Regression Implementation

In [26]:
class LogisticRegressionImpl(LinearClassifier):
    def __init__(self, n_iter=20, lr=0.01, probability=False, regularizer=0.0, print_epoch_stats=True):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lr = lr
        self.probability = probability
        self.regularizer = regularizer
        self.print_epoch_stats = print_epoch_stats

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

       # training algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):
                score = y * x.dot(self.w)
                self.w = (1 - self.regularizer * self.lr) * self.w + self.lr * y * self.sigmoid(-score) * x

            if self.print_epoch_stats:
                print(f'epoch : {i}, train_loss : {self.score(X, Y)}')

    def score(self, X, Y):
        loss = 0.0
        min_weight = np.min(self.w.dot(self.w))
        for x, y in zip(X, self.encode_outputs(Y)):
            # loss for logistic regression
            loss += float(np.log(1 + np.exp(-y * x.dot(self.w))))
        return self.regularizer / 2 * min_weight + loss / X.shape[0]

## Preparing training data

In [27]:

import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


# Read all the documents.
X, Y = read_data('data/all_sentiment_shuffled.txt')

# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)


## Testing our classifiers 	

In [5]:

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our LogisticRegression implementation
    SVCImpl()
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

epoch : 0, train_loss : 0.7057455996879557
epoch : 1, train_loss : 0.5611505980919427
epoch : 2, train_loss : 0.5039397829488059
epoch : 3, train_loss : 0.4696387462877015
epoch : 4, train_loss : 0.4462894936516433
epoch : 5, train_loss : 0.42830267475548656
epoch : 6, train_loss : 0.4143419948195836
epoch : 7, train_loss : 0.40286881739702346
epoch : 8, train_loss : 0.39358083068490124
epoch : 9, train_loss : 0.3857290503206957
epoch : 10, train_loss : 0.37876299806577396
epoch : 11, train_loss : 0.3726294761723526
epoch : 12, train_loss : 0.3671505317718635
epoch : 13, train_loss : 0.3621148454079944
epoch : 14, train_loss : 0.3574758489887846
epoch : 15, train_loss : 0.3532854985167091
epoch : 16, train_loss : 0.34951368393329507
epoch : 17, train_loss : 0.346134213194824
epoch : 18, train_loss : 0.3430387742790991
epoch : 19, train_loss : 0.340125666100796
Training time: 4.74 sec.
Accuracy: 0.8363.


In [6]:

# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our LogisticRegression implementation
    LogisticRegressionImpl()
)

# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print('Training time: {:.2f} sec.'.format(t1-t0))

# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

epoch : 0, train_loss : 0.6288857169774531
epoch : 1, train_loss : 0.5840018940929901
epoch : 2, train_loss : 0.5511767701367489
epoch : 3, train_loss : 0.5261244113878396
epoch : 4, train_loss : 0.5062936920256184
epoch : 5, train_loss : 0.49011851663793615
epoch : 6, train_loss : 0.4765995710252395
epoch : 7, train_loss : 0.4650742894372502
epoch : 8, train_loss : 0.45508760073903537
epoch : 9, train_loss : 0.4463169192656121
epoch : 10, train_loss : 0.4385270914760717
epoch : 11, train_loss : 0.43154242018454747
epoch : 12, train_loss : 0.4252287470781953
epoch : 13, train_loss : 0.41948164728788945
epoch : 14, train_loss : 0.4142184477857359
epoch : 15, train_loss : 0.4093727030957251
epoch : 16, train_loss : 0.4048902894709579
epoch : 17, train_loss : 0.40072658932183824
epoch : 18, train_loss : 0.3968444253813822
epoch : 19, train_loss : 0.3932125202979789
Training time: 4.23 sec.
Accuracy: 0.8175.


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

param_grid = {
    'classifier__n_iter': [10, 30, 10],
    'classifier__regularizer': [0.1, 1, 2]
}


pipeline = Pipeline(steps=[
    ("vect",TfidfVectorizer()),
    ("select", SelectKBest(k=1000)),
    ("norm", Normalizer()),
    ("classifier", SVCImpl(print_epoch_stats=False))
])

grid_search = GridSearchCV(pipeline, param_grid, n_jobs=-1, return_train_score=True)

grid_search.fit(X, Y)

print("Best parameter (loss=%0.3f):" % grid_search.best_score_)
print(grid_search.best_params_)

## Optimizing Performance

### a) BLAS operations

In [60]:
from scipy.linalg.blas import ddot, dscal, daxpy

class SVCImplBLAS(LinearClassifier):
    def __init__(self, n_iter=20, lr=0.01, regularizer=0.0, print_epoch_stats=True):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter
        self.lr = lr
        self.regularizer = regularizer
        self.print_epoch_stats = print_epoch_stats

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # training algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = ddot(self.w, x)
                
                dscal(1 - self.regularizer * self.lr, self.w)
                if y * score < 1:
                    daxpy(x, self.w, a=(self.lr * y))
                
            if self.print_epoch_stats:
                print(f'epoch : {i}, train_loss : {self.score(X, Y)}')

    def score(self, X, Y):
        loss = 0.0
        min_weight = np.min(ddot(self.w, self.w))
        for x, y in zip(X, self.encode_outputs(Y)):
            # loss for svc
            loss += max(0.0, 1.0 - y * x.dot(self.w))
        return self.regularizer / 2 * min_weight + loss / X.shape[0]

### Performance Evaluation

In [66]:
def evaluate_pipeline(pipeline):
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

#### Baseline

In [67]:
%%timeit
pipeline_baseline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our LogisticRegression implementation
    SVCImpl(print_epoch_stats=False)
)
pipeline_baseline.fit(Xtrain, Ytrain)

2.37 s ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
evaluate_pipeline(pipeline_baseline)

NotFittedError: The TF-IDF vectorizer is not fitted

#### BLAS

In [61]:
%%timeit

pipeline_blas = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),

    # NB that this is our LogisticRegression implementation
    SVCImplBLAS(print_epoch_stats=False)
)
pipeline_blas.fit(Xtrain, Ytrain)

2.66 s ± 131 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
evaluate_pipeline(pipeline_blas)

Accuracy: 0.8363.
